In [1]:
!pip install -q pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install -q torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q unsloth sentence_transformers

dill 0.3.8 is installed but dill<0.3.2,>=0.3.1.1 is required
Redoing requirement with just package name...
cloudpickle 3.1.0 is installed but cloudpickle~=2.2.1 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<1.25.0,>=1.14.3 is required
Redoing requirement with just package name...
pyarrow 17.0.0 is installed but pyarrow<10.0.0,>=3.0.0 is required
Redoing requirement with just package name...
jupyterlab 4.3.1 is installed but jupyterlab~=3.6.0 is required
Redoing requirement with just package name...
google-cloud-bigquery 2.34.4 is installed but google-cloud-bigquery[bqstorage,pandas]>=3.10.0 is required
Redoing requirement with just package name...
google-cloud-storage 1.44.0 is installed but google-cloud-storage>=2.0.0 is required
Redoing requirement with just package name...
pandas 2.2.3 is installed but pandas<2.1.4,>=1.5.0 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<3.0.0,>=2.0.0 is req

In [2]:
import numpy as np
import pandas as pd 
import os
from datasets import Dataset
from tqdm import tqdm
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from kaggle_secrets import UserSecretsClient
import wandb
import os

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import numpy as np
import pandas as pd 
import os

n_train_samples = 4369#3000, 4370 is the max
n_test_samples = 1#1000
add_synth = False
add_malalgo = False
add_missing_filled = False

In [4]:
# Load data
DATA_PATH = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
train = pd.read_csv(DATA_PATH+"/train.csv")
mis_map = pd.read_csv(DATA_PATH+"/misconception_mapping.csv")
eedi_synth = pd.read_csv("/kaggle/input/eedi-sythetic-data/eedi_sythetic.csv")
malalgoqa = pd.read_csv("/kaggle/input/malalgoqa/MalAlgoQA.csv")
missing_filled = pd.read_csv("/kaggle/input/eedi-missing-filled/misconception_mapping_filled.csv")

In [5]:
#  misconceptions are tagged to which subject and/or construct
train_eda_answer_long = train.melt(id_vars=['QuestionId','ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'QuestionText', 'CorrectAnswer'],
                            value_vars=['AnswerAText','AnswerBText', 'AnswerCText', 'AnswerDText'],
                           var_name="Answer",
                           value_name="AnswerText")


train_eda_misconception_long = train.melt(id_vars=['QuestionId','ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'QuestionText', 'CorrectAnswer'],
                            value_vars=['MisconceptionAId','MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId'],
                           var_name="Misconception",
                           value_name="MisconceptionId")
train_eda_long = train_eda_answer_long.copy()
train_eda_long['MisconceptionId'] = train_eda_misconception_long['MisconceptionId']
train_eda_long=train_eda_long.sort_values(['QuestionId'])

# Pulling in the misconception
train_eda_long['AnswerOption'] = train_eda_long['Answer'].str[6:7]
train_misconceptions = train_eda_long[train_eda_long['AnswerOption'] != train_eda_long['CorrectAnswer']].copy().reset_index(drop=True).drop(columns=['Answer'])
train_misconceptions = train_misconceptions.merge(mis_map, how='left', on='MisconceptionId')

In [6]:
# Processing for Eedi synthetic dataset
eedi_synth = eedi_synth.drop(columns=['CorrectAnswerText']).rename(columns={"MisconceptionText":"MisconceptionName",
                                                                            "MisconceptionID":"MisconceptionId"})
eedi_synth['Q&A'] = "Math question: " + eedi_synth['QuestionText'] + "\n" + "Wrong Answer: " + eedi_synth['AnswerText']


In [7]:
# Process malalgoqa
malalgoqa = malalgoqa.rename(columns={"Answer":"CorrectAnswer",
                         "Question":"QuestionText"})
malalgoqa_answer_long = malalgoqa.melt(id_vars=['ID','QuestionText', "CorrectAnswer"],
                            value_vars=['Choice_A','Choice_B', 'Choice_C', 'Choice_D'],
                           var_name="Answer",
                           value_name="AnswerText")

malalgoqa_misconception_long = malalgoqa.melt(id_vars=['ID','QuestionText', "CorrectAnswer"],
                            value_vars=['Rationale_A','Rationale_B', 'Rationale_C', 'Rationale_D'],
                           var_name="Misconception",
                           value_name="MisconceptionName")
# Pulling in the misconception
malalgoqa_answer_long['MisconceptionName'] = malalgoqa_misconception_long['MisconceptionName']
malalgoqa_answer_long = malalgoqa_answer_long.sort_values(['ID'])
malalgoqa_answer_long

# Remove correct answers
malalgoqa_answer_long['AnswerOption'] = malalgoqa_answer_long['Answer'].str[-1]
malalgoqa_answer_long = malalgoqa_answer_long[malalgoqa_answer_long['AnswerOption'] != malalgoqa_answer_long['CorrectAnswer']].copy().reset_index(drop=True).drop(columns=['Answer'])
#malalgoqa_answer_long.groupby('ID')['QuestionText'].count().min()

malalgoqa_answer_long['Q&A'] = "Math question: " + malalgoqa_answer_long['QuestionText'] + "\n" + "Wrong Answer: " + malalgoqa_answer_long['AnswerText']
malalgoqa_answer_long = malalgoqa_answer_long[malalgoqa_answer_long['MisconceptionName'].notnull()].reset_index(drop=True)
malalgoqa_answer_long

,ID,QuestionText,CorrectAnswer,AnswerText,MisconceptionName,AnswerOption,Q&A
0,0,Which list shows the following number in order from highest to lowest?,C,235 237 254 276,Ordered least to greatest,A,Math question: Which list shows the following number in order from highest to lowest?\nWrong Answer: 235 237 254 276
1,0,Which list shows the following number in order from highest to lowest?,C,237 276 235 254,Ordered greatest to least by ones place.,B,Math question: Which list shows the following number in order from highest to lowest?\nWrong Answer: 237 276 235 254
2,0,Which list shows the following number in order from highest to lowest?,C,276 254 235 237,Switched last 2 numbers.,D,Math question: Which list shows the following number in order from highest to lowest?\nWrong Answer: 276 254 235 237
3,1,"Which is another way to represent 2,819?",B,"2,000 + 800 + 100 + 90",Misrepresents the place value of 1 as hundreds instead of tens,D,"Math question: Which is another way to represent 2,819?\nWrong Answer: 2,000 + 800 + 100 + 90"
4,1,"Which is another way to represent 2,819?",B,"2,000 + 8,000 + 10 + 9",Misrepresents the place value of 8 as thousands instead of hundreds.,C,"Math question: Which is another way to represent 2,819?\nWrong Answer: 2,000 + 8,000 + 10 + 9"
...,...,...,...,...,...,...,...
2416,805,"A fish tank shaped like a rectangular prism is 78 inches tall, 66 inches wide, and 2 feet long. One cubic foot of water weighs approximately 62.43 pounds. What is the weight of the water the tank can hold when it is full?",C,164.9 pounds,"Did not use the conversion factor with the 2 dimensions. Calculated the volume in cubic feet, then divided by 62.43.",B,"Math question: A fish tank shaped like a rectangular prism is 78 inches tall, 66 inches wide, and 2 feet long. One cubic foot of water weighs approximately 62.43 pounds. What is the weight of the water the tank can hold when it is full?\nWrong Answer: 164.9 pounds"
2417,805,"A fish tank shaped like a rectangular prism is 78 inches tall, 66 inches wide, and 2 feet long. One cubic foot of water weighs approximately 62.43 pounds. What is the weight of the water the tank can hold when it is full?",C,"642,779 pounds",Multiplied all of the numbers provided.,D,"Math question: A fish tank shaped like a rectangular prism is 78 inches tall, 66 inches wide, and 2 feet long. One cubic foot of water weighs approximately 62.43 pounds. What is the weight of the water the tank can hold when it is full?\nWrong Answer: 642,779 pounds"
2418,806,"The total number of books sold by an online bookstore was at least 18 more than two-thirds of the number of books sold in their physical store. If the online bookstore sold precisely 246 books, what inequality can be used to calculate x, representing the number of books sold in the physical store?",A,(2/3) (246) + 18 ≤ x,Calculated 2/3 of the incorrect value.,C,"Math question: The total number of books sold by an online bookstore was at least 18 more than two-thirds of the number of books sold in their physical store. If the online bookstore sold precisely 246 books, what inequality can be used to calculate x, representing the number of books sold in the physical store?\nWrong Answer: (2/3) (246) + 18 ≤ x"
2419,806,"The total number of books sold by an online bookstore was at least 18 more than two-thirds of the number of books sold in their physical store. If the online bookstore sold precisely 246 books, what inequality can be used to calculate x, representing the number of books sold in the physical store?",A,(2/3) (246) + 18 ≥ x,"Calculated 2/3 of the incorrect value and misinterpreted ""at least.""",D,"Math question: The total number of books sold by an online bookstore was at least 18 more than two-thirds of the number of books sold in their physical store. If the online bookstore sold precisely 246 books, what inequality can be used to calculate x, representing the number of books sold in the physical store?\nWrong Answer: (2/3) (246) + 18 ≥ x"


In [8]:
# Processing filled missing misconceptions
mis_map_for_filling = mis_map.copy()
mis_map_for_filling['MisconceptionId'] = mis_map_for_filling['MisconceptionId'].astype(str)

missing_filled['MisconceptionName'] = missing_filled['MisconceptionName'].str[:-1]
missing_filled = missing_filled.merge(mis_map_for_filling, how='left', left_on='MisconceptionName', 
                     right_on='MisconceptionId', suffixes=('', '_tofill'))
missing_filled['MisconceptionName'] = np.where(missing_filled['MisconceptionName_tofill'].notnull(),
                                              missing_filled['MisconceptionName_tofill'], missing_filled['MisconceptionName'])
missing_filled['Q&A'] = "Math question: " + missing_filled['QuestionText'] + "\n" + "Wrong Answer: " + missing_filled['AnswerText']
missing_filled = missing_filled.drop(columns=['RowIndex', 'MisconceptionId', 'MisconceptionName_tofill'])

In [9]:
# Split into those with NA and those without
train_misconceptions_noNA = train_misconceptions[train_misconceptions['MisconceptionId'].notnull()].copy().reset_index(drop=True)
train_misconceptions_NA = train_misconceptions[train_misconceptions['MisconceptionId'].isnull()].copy().reset_index(drop=True)

# test on train_sample to see inference output
train_misconceptions_noNA['Q&A'] = "Math question: " + train_misconceptions_noNA['QuestionText'] + "\n" + "Wrong Answer: " + train_misconceptions_noNA['AnswerText']
train_sample = (train_misconceptions_noNA
                .sample(n_train_samples,random_state=42)
               )
test_sample = (train_misconceptions_noNA[~train_misconceptions_noNA["Q&A"].isin(train_sample["Q&A"])]
                .sample(n_test_samples,random_state=42)
              ) 


In [10]:
# # Checcking on missing filled for those with no NAs
# chk_missing_filled = pd.concat([train_misconceptions_noNA, missing_filled],axis=0, ignore_index=True).sort_values('QuestionText')
# sampled_questions = chk_missing_filled['QuestionText'].sample(20)
# chk_missing_filled[chk_missing_filled['QuestionText'].isin(sampled_questions)].to_csv('sampled_filled_qns_compare.csv', index=False)


In [11]:
# Adding on extra datasets to train_sample. Should not 

# Merge if synthetic
if add_synth:
    print("Adding synthetic data")
    train_sample = pd.concat([train_sample, eedi_synth], axis=0, ignore_index=True, sort=False)

# Merge if want malalgo in training set
if add_malalgo:
    print("Adding malalgo data")
    train_sample = pd.concat([train_sample, malalgoqa_answer_long], axis=0, ignore_index=True, sort=False)

if add_missing_filled:
    print("Adding filled missing misconception data")
    train_sample = pd.concat([train_sample, missing_filled], axis=0, ignore_index=True, sort=False)


assert train_sample['MisconceptionName'].isnull().sum() == 0, "Missing misconception name in data"

In [12]:
# Load model
print("Loading model")
torch.cuda.empty_cache()
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-14B", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Loading model
==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/210k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Inference on test

In [13]:
# Specify prompt template
print("Running inference on test before finetuning")
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

# Inference time
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!

output_list=[]
for idx in tqdm(range(test_sample.shape[0])):
    if idx%50 == 0:
        print("Before finetuning idx: ",idx)
    inputs = tokenizer(
    [
        prompt_style.format(
            """You are a math teacher. 
            A student selected the wrong answer to a math question. 
            Identify what mathematical concept the student does not understand.           
            """, # instruction
            test_sample['Q&A'].to_list()[idx], # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    
    outputs = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, 
                             max_new_tokens = 128, use_cache = True)
    outputs_decoded = tokenizer.batch_decode(outputs)
    output_list.append(outputs_decoded[0])

# Output
col_name = 'llm_response_prompt_before_finetuning'
test_sample[col_name] = output_list
test_sample[col_name] = test_sample[col_name].str.split("Response:\n",expand=True,n=2)[1].str.replace('<|endoftext|>', '')


Running inference on test before finetuning


  0%|          | 0/1 [00:00<?, ?it/s]

Before finetuning idx:  0


100%|██████████| 1/1 [00:13<00:00, 13.10s/it]


# Fine-tuning

In [14]:
# Setting PEFT adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128. The higher the rank, the more parameters to train, but the more complex details are captured. Lowering this from 128 to 64 results in slower loss going to 0
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # This affects how much to adjust the LORA weights by. Setting r to 128 and alpha to 256 made the loss higher with training 
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [15]:
# Loading and processing the dataset
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a math teacher. A student selected the wrong answer to a math question. Identify what mathematical concept the student does not understand.  

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Q&A"] 
    outputs      = examples["MisconceptionName"] 
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_pandas(train_sample)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][0]

Map:   0%|          | 0/4369 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a math teacher. A student selected the wrong answer to a math question. Identify what mathematical concept the student does not understand.  \n\n### Input:\nMath question: What sign would make this statement true?\n\\(\n\\frac{4}{6} ? \\frac{10}{15}\n\\)\nWrong Answer: \\( < \\)\n\n### Response:\nBelieves that fractions with larger denominators are greater in size<|endoftext|>'

In [16]:
# Setting hyperparamters for the trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 32,
        
        num_train_epochs = 3,# Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = -1,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/4369 [00:00<?, ? examples/s]

In [17]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.172 GB of memory reserved.


In [18]:
print("fine-tuning model")
trainer_stats = trainer.train()

fine-tuning model


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 32
\        /    Total batch size = 32 | Total steps = 408
 "-____-"     Number of trainable parameters = 275,251,200


Step,Training Loss
10,1.240000
20,0.670400
30,0.625500
40,0.572100
50,0.524400
60,0.480300
70,0.466200
80,0.457900
90,0.401300
100,0.393600


In [19]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

30076.1151 seconds used for training.
501.27 minutes used for training.
Peak reserved memory = 14.521 GB.
Peak reserved memory for training = 3.349 GB.
Peak reserved memory % of max memory = 98.508 %.
Peak reserved memory for training % of max memory = 22.719 %.


In [20]:
model.save_pretrained("finetuned_lora")

# Test inference

In [21]:
# Specify prompt template
print("Running inference on test after finetuning")
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

# Inference time
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!

output_list=[]
for idx in tqdm(range(test_sample.shape[0])):
    if idx%50 == 0:
        print("After finetuning idx: ", idx)
    inputs = tokenizer(
    [
        prompt_style.format(
            """You are a math teacher. 
            A student selected the wrong answer to a math question. 
            Identify what mathematical concept the student does not understand.           
            """, # instruction
            test_sample['Q&A'].to_list()[idx], # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    
    outputs = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, 
                             max_new_tokens = 128, use_cache = True)
    outputs_decoded = tokenizer.batch_decode(outputs)
    output_list.append(outputs_decoded[0])

# Output
col_name = 'llm_response_prompt_after_finetuning'
test_sample[col_name] = output_list
test_sample[col_name] = test_sample[col_name].str.split("Response:\n",expand=True,n=2)[1].str.replace('<|endoftext|>', '')


Running inference on test after finetuning


  0%|          | 0/1 [00:00<?, ?it/s]

After finetuning idx:  0


100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [22]:
# Save output
test_sample.to_csv('test_sample.csv',index=False)
test_sample

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,AnswerText,MisconceptionId,AnswerOption,MisconceptionName,Q&A,llm_response_prompt_before_finetuning,llm_response_prompt_after_finetuning
860,357,3132,Write algebraic expressions with correct algebraic convention,70,Writing Expressions,"Which of the following is the correct way of writing:\n\n\( m \) multiplied by \( n \), then add \( 5 \)\n\nusing algebraic convention?",C,\( m \times n+5 \),1680.0,B,Does not realize we can omit the multiplication symbol in algebraic expressions,"Math question: Which of the following is the correct way of writing:\n\n\( m \) multiplied by \( n \), then add \( 5 \)\n\nusing algebraic convention?\nWrong Answer: \( m \times n+5 \)","\nThe student does not understand the order of operations in algebraic expressions. Specifically, they did not apply the correct order of operations (PEMDAS/BODMAS) to the expression, which states that multiplication and division should be performed before addition and subtraction. The correct answer should be \( mn + 5 \), where \( mn \) represents the product of \( m \) and \( n \).",\nDoes not realize we can omit the multiplication symbol in algebraic expressions


# Similarity search

In [23]:
# For misconceptions
model = SentenceTransformer("all-mpnet-base-v2")
misconceptions = mis_map["MisconceptionName"].to_list()
misconception_embeddings = model.encode(misconceptions)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [24]:
# Supporting functions
def find_most_similar_misconception(generated_misconception, misconception_embeddings, misconceptions):
    # Encode the generated misconception
    gen_misconception_embedding = model.encode([generated_misconception])
    
    # Calculate cosine similarity between the generated misconception and all known misconceptions
    similarities = cosine_similarity(gen_misconception_embedding, misconception_embeddings)[0]
    
    # Get indices of misconceptions sorted by similarity
    sorted_indices = np.argsort(similarities)[::-1]
    
    # Get sorted misconceptions and similarity scores
    sorted_misconceptions = [(misconceptions[i], similarities[i]) for i in sorted_indices]
    
    return sorted_misconceptions

def find_id(list_misconception):
    string_id=""
    i=0
    for misc in list_misconception:
        i+=1
        ind=misconceptions.index(misc[0])
        if i==len(list_misconception):
            string_id=string_id+str(ind)
        else:
            string_id=string_id+str(ind)+" "
    return string_id

In [25]:
# Find misconception to LLM response -- Before
def top_k_misconception_id(df, llm_response_col, k):
    MisconceptionId=[]
    for i in range(len(df)):
        simil_misc=find_most_similar_misconception(df[llm_response_col].tolist()[i], misconception_embeddings, misconceptions)
        MisconceptionId.append(find_id(simil_misc[:k]))
    return MisconceptionId

test_sample['MisconceptionId_top25_before_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_before_finetuning", 25)
test_sample['MisconceptionId_top10_before_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_before_finetuning", 10)
test_sample['MisconceptionId_top5_before_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_before_finetuning", 5)

# After
test_sample['MisconceptionId_top25_after_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_after_finetuning", 25)
test_sample['MisconceptionId_top10_after_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_after_finetuning", 10)
test_sample['MisconceptionId_top5_after_ft'] = top_k_misconception_id(test_sample, "llm_response_prompt_after_finetuning", 5)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
def calc_topk_score(df, topk_col):
    print("Top k score for :", topk_col)
    score_list = []
    for misc_id, misc_topk in zip(df['MisconceptionId'], df[topk_col]):
        if str(int(misc_id)) in misc_topk.split(" "):
            score_list.append(1)
            
        else:
            score_list.append(0)
            
    print(f"Score: {round(sum(score_list) / len(score_list), 3)} ({sum(score_list)} / {len(score_list)})",  )
    return None

calc_topk_score(test_sample, "MisconceptionId_top5_before_ft")
calc_topk_score(test_sample, "MisconceptionId_top5_after_ft")
print("\n")
calc_topk_score(test_sample, "MisconceptionId_top10_before_ft")
calc_topk_score(test_sample, "MisconceptionId_top10_after_ft")
print("\n")
calc_topk_score(test_sample, "MisconceptionId_top25_before_ft")
calc_topk_score(test_sample, "MisconceptionId_top25_after_ft")

Top k score for : MisconceptionId_top5_before_ft
Score: 0.0 (0 / 1)
Top k score for : MisconceptionId_top5_after_ft
Score: 1.0 (1 / 1)


Top k score for : MisconceptionId_top10_before_ft
Score: 0.0 (0 / 1)
Top k score for : MisconceptionId_top10_after_ft
Score: 1.0 (1 / 1)


Top k score for : MisconceptionId_top25_before_ft
Score: 0.0 (0 / 1)
Top k score for : MisconceptionId_top25_after_ft
Score: 1.0 (1 / 1)
